In [21]:
%run ../Python_files/util_data_storage_and_load.py

In [22]:
%run ../Python_files/load_dicts.py

In [73]:
%run ../Python_files/util.py

In [32]:
road_seg_inr_capac = zload('../temp_files/road_seg_inr_capac.pkz')

In [35]:
# load tmc-day-ave_speed data for AM peak of January
tmc_day_speed_dict_Jan_AM = zload('../temp_files/Jan_AM/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for MD of January
tmc_day_speed_dict_Jan_MD = zload('../temp_files/Jan_MD/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for PM peak of January
tmc_day_speed_dict_Jan_PM = zload('../temp_files/Jan_PM/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for NT of January
tmc_day_speed_dict_Jan_NT = zload('../temp_files/Jan_NT/tmc_day_speed_dict.pkz')

In [47]:
tmc_day_speed_dict_Jan_AM['129N0410818'].ave_speed()

58.940018744142527

In [74]:
tmc_day_capac_flow_minute_dict = {}

for i in range(len(road_seg_inr_capac.tmc)):
    for day in range(32)[1:]:
        tmc = road_seg_inr_capac.tmc[i]
        road_num = road_seg_inr_capac.road_num[i]
        shape_length = road_seg_inr_capac.shape_length[i]
        day = day
        AB_AM_capac = road_seg_inr_capac.AB_AM_capac[i]
        AB_MD_capac = road_seg_inr_capac.AB_MD_capac[i]
        AB_PM_capac = road_seg_inr_capac.AB_PM_capac[i]
        AB_NT_capac = road_seg_inr_capac.AB_NT_capac[i]
        AM_ave_speed = tmc_day_speed_dict_Jan_AM[tmc + str(day)].ave_speed()
        MD_ave_speed = tmc_day_speed_dict_Jan_MD[tmc + str(day)].ave_speed()
        PM_ave_speed = tmc_day_speed_dict_Jan_PM[tmc + str(day)].ave_speed()
        NT_ave_speed = tmc_day_speed_dict_Jan_NT[tmc + str(day)].ave_speed()
        AM_speed_minute = tmc_day_speed_dict_Jan_AM[tmc + str(day)].speed
        MD_speed_minute = tmc_day_speed_dict_Jan_MD[tmc + str(day)].speed
        PM_speed_minute = tmc_day_speed_dict_Jan_PM[tmc + str(day)].speed
        NT_speed_minute = tmc_day_speed_dict_Jan_NT[tmc + str(day)].speed
        tmc_day_capac_flow_minute = RoadSegInrCapacFlowMinute(tmc, road_num, shape_length, day, \
                                                  AB_AM_capac, AB_MD_capac, \
                                                  AB_PM_capac, AB_NT_capac, \
                                                  AM_ave_speed, MD_ave_speed, \
                                                  PM_ave_speed, NT_ave_speed, \
                                                  AM_speed_minute, MD_speed_minute, \
                                                  PM_speed_minute, NT_speed_minute)
        assert(len(tmc_day_capac_flow_minute.AM_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.MD_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.PM_flow_minute()) == 120)
        assert(len(tmc_day_capac_flow_minute.NT_flow_minute()) == 120)
        tmc_day_capac_flow_minute_dict[tmc + str(day)] = tmc_day_capac_flow_minute

In [75]:
#zdump(tmc_day_capac_flow_minute_dict, '../temp_files/tmc_day_capac_flow_minute_dict_Jan.pkz')

In [81]:
tmc_day_capac_flow_minute_dict['129N0437720'].NT_flow_minute

<bound method RoadSegInrCapacFlowMinute.NT_flow_minute of <__main__.RoadSegInrCapacFlowMinute object at 0x7f41b4e43190>>

In [69]:
link_with_capac_list = list(zload('../temp_files/links_with_capac.pkz'))

In [85]:
import numpy as np

link_day_minute_Jan_dict = {}

for day in range(32)[1:]:
    i = 0
    for link_with_capac in link_with_capac_list:
        AM_flow = sum([tmc_day_capac_flow_minute_dict[tmc + str(day)].AM_flow() for tmc in link_with_capac.tmc_set])
        MD_flow = sum([tmc_day_capac_flow_minute_dict[tmc + str(day)].MD_flow() for tmc in link_with_capac.tmc_set])
        PM_flow = sum([tmc_day_capac_flow_minute_dict[tmc + str(day)].PM_flow() for tmc in link_with_capac.tmc_set])
        NT_flow = sum([tmc_day_capac_flow_minute_dict[tmc + str(day)].NT_flow() for tmc in link_with_capac.tmc_set])
        AM_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].AM_flow_minute()) \
                              for tmc in link_with_capac.tmc_set]))
        MD_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].MD_flow_minute()) \
                              for tmc in link_with_capac.tmc_set]))
        PM_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].PM_flow_minute()) \
                              for tmc in link_with_capac.tmc_set]))
        NT_flow_minute = list(sum([np.array(tmc_day_capac_flow_minute_dict[tmc + str(day)].NT_flow_minute()) \
                              for tmc in link_with_capac.tmc_set]))
        link_with_capac_new = Link_with_Free_Flow_Time_Minute(link_with_capac.init_node, link_with_capac.term_node, \
                                                       link_with_capac.tmc_set, \
                                                       link_with_capac.AM_capac, \
                                                       link_with_capac.MD_capac, \
                                                       link_with_capac.PM_capac, \
                                                       link_with_capac.NT_capac, \
                                                       link_with_capac.free_flow_time, \
                                                       link_with_capac.length, \
                                                       AM_flow, MD_flow, PM_flow, NT_flow, \
                                                       AM_flow_minute, MD_flow_minute, \
                                                       PM_flow_minute, NT_flow_minute)
        link_day_minute_Jan_dict['link_' + str(i) + '_' + str(day)] = link_with_capac_new
        i = i + 1

In [86]:
#zdump(link_day_minute_Jan_dict, '../temp_files/link_day_minute_Jan_dict.pkz')

In [93]:
link_day_minute_Jan_dict['link_0_10'].AM_flow

334648.35750727874

In [131]:
link_day_minute_Jan_dict_JSON = {}
for link_idx in range(24):
    for day in range(32)[1:]:   
        key = 'link_' + str(link_idx) + '_' + str(day)
        data = {'link_idx': link_idx, 'day': day, \
             'init_node': link_day_minute_Jan_dict[key].init_node, \
             'term_node': link_day_minute_Jan_dict[key].term_node, \
             'AM_capac': link_day_minute_Jan_dict[key].AM_capac, \
             'MD_capac': link_day_minute_Jan_dict[key].MD_capac, \
             'PM_capac': link_day_minute_Jan_dict[key].PM_capac, \
             'NT_capac': link_day_minute_Jan_dict[key].NT_capac, \
             'free_flow_time': link_day_minute_Jan_dict[key].free_flow_time, \
             'length': link_day_minute_Jan_dict[key].length, \
             'AM_flow': link_day_minute_Jan_dict[key].AM_flow, \
             'MD_flow': link_day_minute_Jan_dict[key].MD_flow, \
             'PM_flow': link_day_minute_Jan_dict[key].PM_flow, \
             'NT_flow': link_day_minute_Jan_dict[key].NT_flow, \
             'AM_flow_minute': link_day_minute_Jan_dict[key].AM_flow_minute, \
             'MD_flow_minute': link_day_minute_Jan_dict[key].MD_flow_minute, \
             'PM_flow_minute': link_day_minute_Jan_dict[key].PM_flow_minute, \
             'NT_flow_minute': link_day_minute_Jan_dict[key].NT_flow_minute}
        link_day_minute_Jan_dict_JSON[key] = data
        
import json

# Writing JSON data
with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'w') as json_file:
    json.dump(link_day_minute_Jan_dict_JSON, json_file)

In [149]:
with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON_ = json.load(json_file)

In [152]:
link_day_minute_Jan_dict_JSON_['link_3_9'] ['AM_flow_minute']

[627015.7581123047,
 632646.2768060619,
 615662.1806283699,
 637923.5083821698,
 620549.7552948801,
 635831.0476736326,
 650903.8453416765,
 636574.2668965197,
 643062.2995542071,
 636196.4467736026,
 636942.4321975559,
 645546.2802063663,
 633027.8383092864,
 639694.0912113492,
 639615.3862674409,
 633031.0803901834,
 634764.3812841601,
 639106.0963555203,
 649285.8732593295,
 641964.3632741203,
 643357.2657133499,
 628704.3764719949,
 649403.4230763162,
 636430.014876315,
 638691.5696175551,
 630408.2308600625,
 629123.5699704948,
 647153.2317230613,
 621106.5989874534,
 635128.6790765708,
 638206.9213310801,
 628770.885266849,
 639646.1094300788,
 633784.3933438949,
 643212.6918724127,
 627345.9796176199,
 645662.9918117793,
 645050.3340600448,
 649435.26840816,
 636799.2277455764,
 646581.666741757,
 640545.6469822824,
 631206.509372983,
 634488.051531418,
 628378.3148839712,
 644541.4842990624,
 649857.9777406272,
 621881.6287428022,
 640298.1404456731,
 632946.3190015944,
 629050